# Configuración del entorno de Python

Las siguiente celda corresponde a la instalación de las dependencias y librerias necesarias para el entrenamiento del modelo:

In [ ]:
# Configuración del entorno
!apt-get -qq install xxd
!pip install pandas numpy matplotlib
!pip install tensorflow==2.0.0-rc1

* <code>!apt-get -qq install xxd:</code> Instala la herramienta xxd que se usa para convertir archivos binarios a formato hexadecimal.
* <code>!pip install pandas numpy matplotlib:</code> Instala las bibliotecas pandas, numpy y matplotlib para el análisis de datos y la visualización.
* <code>!pip install tensorflow==2.0.0-rc1:</code> Instala TensorFlow versión 2.0.0-rc1 para el desarrollo de la red nueronal.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Analizar y preparar los datos

*   Esta sección lee los archivos CSV y los convierte a un formato adecuado para entrenar la red neuronal.
*   Se normalizan los datos de entrada para que estén entre 0 y 1.
* Se crea una matriz codificada "one-hot" para las salidas (gestos).

Adicionalmente se dividen aleatoriamente los pares de entrada y salida para cada clase en conjuntos de datos:

* 70 % para entrenamiento
* 10 % para validación
* 20 % para pruebas.




In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import random

# Fijamos las semillas para reproducibilidad
SEED = 43
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

GESTURES = [
    "ayuda",
    "comoestas",
    "cuanto",
    "donde",
    "explicar",
    "gracias",
    "hola",
    "perdon",
    "porfavor",
    "quepaso"
]

SAMPLES_PER_GESTURE = 70
NUM_GESTURES = len(GESTURES)
ONE_HOT_ENCODED_GESTURES = np.eye(NUM_GESTURES)

# Listas globales para train, val, test de TODAS las clases
inputs_train = []
outputs_train = []
inputs_validation = []
outputs_validation = []
inputs_test = []
outputs_test = []

# Proporciones de split dentro de cada clase
TRAIN_RATIO = 0.7
VAL_RATIO = 0.1
TEST_RATIO = 0.2

for gesture_index in range(NUM_GESTURES):
    gesture = GESTURES[gesture_index]

    print(f"Procesando la clase {gesture_index} -> '{gesture}'")

    # Leemos el CSV de la clase
    df = pd.read_csv("/content/drive/MyDrive/Maestria/" + gesture + ".csv")

    # Calculamos cuántas "grabaciones" hay
    num_recordings = df.shape[0] // SAMPLES_PER_GESTURE
    print(f"\tSe encontraron {num_recordings} grabaciones para la clase '{gesture}'.")

    # Codificación one-hot de la clase actual
    output_class = ONE_HOT_ENCODED_GESTURES[gesture_index]

    # Listas LOCALES para la clase actual (antes de hacer el split)
    gesture_inputs = []
    gesture_outputs = []

    # Recolectamos todas las grabaciones para la clase
    for i in range(num_recordings):
        tensor = []
        for j in range(SAMPLES_PER_GESTURE):
            idx = i * SAMPLES_PER_GESTURE + j
            tensor += [
                (df['aX'][idx] + 4) / 8,
                (df['aY'][idx] + 4) / 8,
                (df['aZ'][idx] + 4) / 8,
                (df['gX'][idx] + 2000) / 4000,
                (df['gY'][idx] + 2000) / 4000,
                (df['gZ'][idx] + 2000) / 4000,
                df['Angle1'][idx] / 90,
                df['Angle2'][idx] / 90,
                df['Angle3'][idx] / 90,
                df['Angle4'][idx] / 90,
                df['Angle5'][idx] / 90
            ]
        gesture_inputs.append(tensor)
        gesture_outputs.append(output_class)

    # Convertimos a arrays (aunque para shuffle podemos usar listas)
    gesture_inputs = np.array(gesture_inputs)
    gesture_outputs = np.array(gesture_outputs)

    # Mezclamos (aleatorizamos) SOLO dentro de la clase actual
    combined = list(zip(gesture_inputs, gesture_outputs))
    random.shuffle(combined)
    gesture_inputs, gesture_outputs = zip(*combined)
    gesture_inputs = np.array(gesture_inputs)
    gesture_outputs = np.array(gesture_outputs)

    # Hacemos el split en TRAIN, VAL, TEST (70/10/20)
    total_gesture_samples = len(gesture_inputs)
    train_size = int(TRAIN_RATIO * total_gesture_samples)
    val_size = int(VAL_RATIO * total_gesture_samples)
    # El resto para test
    test_size = total_gesture_samples - train_size - val_size

    # Divisiones para la clase actual
    gesture_inputs_train = gesture_inputs[:train_size]
    gesture_outputs_train = gesture_outputs[:train_size]

    gesture_inputs_val = gesture_inputs[train_size:train_size + val_size]
    gesture_outputs_val = gesture_outputs[train_size:train_size + val_size]

    gesture_inputs_test = gesture_inputs[train_size + val_size:]
    gesture_outputs_test = gesture_outputs[train_size + val_size:]

    # Agregamos a las listas GLOBAL (para TODAS las clases)
    inputs_train.extend(gesture_inputs_train)
    outputs_train.extend(gesture_outputs_train)

    inputs_validation.extend(gesture_inputs_val)
    outputs_validation.extend(gesture_outputs_val)

    inputs_test.extend(gesture_inputs_test)
    outputs_test.extend(gesture_outputs_test)

# Finalmente, convertimos a arrays de NumPy
inputs_train = np.array(inputs_train)
outputs_train = np.array(outputs_train)
inputs_validation = np.array(inputs_validation)
outputs_validation = np.array(outputs_validation)
inputs_test = np.array(inputs_test)
outputs_test = np.array(outputs_test)

print("\nCompletado el parsing y la preparación de TODOS los datos.")
print("Tamaño de inputs_train:", inputs_train.shape)
print("Tamaño de outputs_train:", outputs_train.shape)
print("Tamaño de inputs_validation:", inputs_validation.shape)
print("Tamaño de outputs_validation:", outputs_validation.shape)
print("Tamaño de inputs_test:", inputs_test.shape)
print("Tamaño de outputs_test:", outputs_test.shape)

# Entrenamiento de la Red Neuronal





## Construir y entrenar el modelo

* El modelo cuenta con tres capas densas.
* Se compila el modelo con el optimizador 'rmsprop' y la función de pérdida 'categorical_crossentropy', teniendo como métrica de evaluación la precisión.

In [ ]:
# Construir el modelo y entrenarlo
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(15, activation='relu'))
model.add(tf.keras.layers.Dense(NUM_GESTURES, activation='softmax')) # Se utiliza softmax porque solo esperamos que ocurra un gesto por entrada

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

history = model.fit(inputs_train, outputs_train, epochs=30, batch_size=1, validation_data=(inputs_validate, outputs_validate))

## Etapa de verificación

En esta sección se gráfica la pérdida del modelo durante el entrenamiento, para el conjunto de entrenamiento y el conjunto de validación:


### Pérdida

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)

# Obtener la pérdida (loss) del entrenamiento y la validación desde el historial
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

# Graficar la pérdida del entrenamiento y la validación
plt.plot(epochs, loss, 'g.', label='Pérdida de entrenamiento')
plt.plot(epochs, val_loss, 'b', label='Pérdida de validación')
plt.title('Pérdida de entrenamiento y validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

### Ejecución con datos de prueba
Aqui se la prueba del modelo con los datos de prueba nunca antes vistos por el modelos, se inicia realizando las predicciones:

In [ ]:
# Usar el modelo para predecir las entradas de prueba
predictions = model.predict(inputs_test)

# Imprimir las predicciones y las salidas esperadas
print("Predicciones =\n", np.round(predictions, decimals=3))
print("Reales =\n", outputs_test)

Posteriormente se gráfica la matriz de confusión y el reporte de clasificación:

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,classification_report
import numpy as np

# Obtener las predicciones y convertirlas a las clases predichas (argmax para clasificación)
predicted_classes = np.argmax(predictions, axis=1)  # Convertir probabilidades a clases
true_classes = np.argmax(outputs_test, axis=1)      # Convertir etiquetas one-hot a clases

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(true_classes, predicted_classes)

# Mostrar la matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
disp.plot(cmap='viridis')  # Puedes cambiar el colormap si lo deseas
plt.title("Matriz de Confusión")
plt.show()

print("\nReporte de clasificación:\n")
print(classification_report(y_test, y_pred))

# Convertir el modelo entrenado a TensorFlow Lite

La siguiente celda convierte el modelo al formato TFlite. También se imprime el tamaño en bytes del modelo para asegurar que el tamaño funcione dentro del arduino.

In [ ]:
# Convertir el modelo al formato TensorFlow Lite sin cuantización
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Guardar el modelo en disco
open("gesture_model.tflite", "wb").write(tflite_model)

import os
basic_model_size = os.path.getsize("gesture_model.tflite")
print("El modelo tiene un tamaño de %d bytes" % basic_model_size)

## Codificar el modelo en un archivo de encabezado de Arduino

La siguiente celda crea una matriz de bytes constantes que contiene el modelo TFlite. Se importa el modelo para ser usado por Arduino:

In [ ]:
!echo "const unsigned char model[] = {" > /content/model.h
!cat gesture_model.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"El archivo de cabecera, model.h, tiene un tamaño de {model_h_size:,} bytes.")